# Azure AI Search Agentic retrieval with Reflection Search

Azure AI Search では、Agentic retrieval ではコンテキストとユーザーの質問を使用して、 ナレッジ ソース内のコンテンツに対して実行できるさまざまなサブクエリを生成します。 ナレッジ ソースは、Azure AI Search のインデックス付きコンテンツ、またはプロバイダーにネイティブな API を使用して取得されたリモート コンテンツを指すことができます。 

Azure AI Search client library for Python ライブラリは `11.7.0b2` 以降が必要です。

In [ ]:
!pip install azure-search-documents==11.7.0b2

## Azure AI Search への接続とインデックスの確認

インデックスは既に存在する想定です。
 
- Agentic retrieval 用のインデックスを作成する
    - https://learn.microsoft.com/azure/search/agentic-retrieval-how-to-create-index

In [ ]:
from azure.search.documents import SearchIndexingBufferedSender
from azure.search.documents.knowledgebases import KnowledgeBaseRetrievalClient
from azure.search.documents.knowledgebases.models import KnowledgeBaseRetrievalRequest, KnowledgeBaseMessage, KnowledgeBaseMessageTextContent, SearchIndexKnowledgeSourceParams
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import SearchIndex, SearchField, VectorSearch, VectorSearchProfile, HnswAlgorithmConfiguration, AzureOpenAIVectorizer, AzureOpenAIVectorizerParameters, SemanticSearch, SemanticConfiguration, SemanticPrioritizedFields, SemanticField, SearchIndexKnowledgeSource, SearchIndexKnowledgeSourceParameters, SearchIndexFieldReference, KnowledgeBase, KnowledgeBaseAzureOpenAIModel, KnowledgeSourceReference, KnowledgeRetrievalOutputMode, KnowledgeRetrievalLowReasoningEffort, KnowledgeRetrievalMediumReasoningEffort

from azure.core.credentials import AzureKeyCredential  
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
import os

load_dotenv()

endpoint = os.environ['SEARCH_ENDPOINT']
index_name = os.environ['SEARCH_INDEX_NAME']
credential = DefaultAzureCredential()
token_provider = get_bearer_token_provider(credential, "https://search.azure.com/.default")
#credential = AzureKeyCredential(os.environ['SEARCH_API_KEY'])

#index_client = SearchIndexClient(endpoint=endpoint, credential=credential)
index_client = SearchIndexClient(endpoint=endpoint, credential=credential, token_provider=token_provider)

#Check: get index
try:
    index = index_client.get_index(index_name)
    print(f"Index '{index_name}' already exists.")
except Exception as e:
    print(f"Index '{index_name}' does not exist. Creating a new index.")
   

## ナレッジベースとナレッジソースの設定

In [ ]:
knowledge_source_name = "wikipedia-knowledge-source"
knowledge_base_name = "wikipedia-knowledge-base"

## ナレッジソースの作成
この手順では、以前に作成したインデックスを対象とするナレッジソースを作成します。インデックスと 1:1 対応します。さらにインデックスだけでなく対応する外部・内部データソースも設定できます。

https://learn.microsoft.com/azure/search/agentic-knowledge-source-overview

In [ ]:

# Create a knowledge source
ks = SearchIndexKnowledgeSource(
    name=knowledge_source_name,
    description="Wikipedia の知識ソース",
    search_index_parameters=SearchIndexKnowledgeSourceParameters(
        search_index_name=index_name,
        source_data_fields=[SearchIndexFieldReference(name="docid"), SearchIndexFieldReference(name="text")]
    ),
)

index_client = SearchIndexClient(endpoint=endpoint, credential=credential)
index_client.create_or_update_knowledge_source(knowledge_source=ks)
print(f"Knowledge source '{knowledge_source_name}' created or updated successfully.")

### ナレッジソースの取得

In [ ]:
knowledge_source = index_client.get_knowledge_source(knowledge_source_name)
knowledge_source.serialize()

## ナレッジベースの作成
このステップではナレッジベースを作成します。これはナレッジソースと LLM をラップする役割を果たします。
`EXTRACTIVE_DATA` は既定で生成的な変更を加えずにナレッジソースからコンテンツを返します。Foundry Agent Serviceとの連携にはこれが推奨されます。

https://learn.microsoft.com/azure/search/agentic-retrieval-how-to-create-knowledge-base?tabs=rbac&pivots=python


- 引用に基づく応答に回答合成を使用する
    - https://learn.microsoft.com/azure/search/agentic-retrieval-how-to-answer-synthesis


In [ ]:

#  Create a knowledge base
aoai_params = AzureOpenAIVectorizerParameters(
    resource_url=os.environ['AOAI_ENDPOINT'],
    deployment_name=os.environ['AOAI_DEPLOYMENT_NAME'],
    model_name=os.environ['AOAI_MODEL_NAME'],
)

knowledge_base = KnowledgeBase(
    name=knowledge_base_name,
    description = "This knowledge base handles questions directed at two unrelated sample indexes.",
    models=[KnowledgeBaseAzureOpenAIModel(azure_open_ai_parameters=aoai_params)],
    knowledge_sources=[
        KnowledgeSourceReference(
            name=knowledge_source_name
        )
    ],
    output_mode=KnowledgeRetrievalOutputMode.EXTRACTIVE_DATA, # Choose ANSWER_SYNTHESIS or EXTRACTIVE_DATA
    answer_instructions="質問に対して適切な回答を提供してください。"
)

index_client.create_or_update_knowledge_base(knowledge_base)
print(f"Knowledge base '{knowledge_base_name}' created or updated successfully.")

### ナレッジベースの取得

In [ ]:
knowledge_base = index_client.get_knowledge_base(knowledge_base_name)
knowledge_base.serialize()

### Messages の定義
メッセージは Agentic retireval の入力であり、会話履歴を含みます。各メッセージには、その発信元を示すrole（例：`system` または `user` ）と、自然言語による `content` が含まれます。使用する LLM によって有効な役割が決まります。

In [ ]:
instructions = """
あなたはあらゆる質問に回答できるFAQエージェント。
ソースはJSON形式で、回答内に引用する必要があるref_idを含んでいます。
回答が分からない場合は「分かりません」と回答してください。
"""

messages = [
    {
        "role": "system",
        "content": instructions
    }
]

シンプルなクエリのみの場合上記は不要です。

## コンテンツの検索
このステップでは Agentic retrieval パイプラインを実行し、根拠に基づく引用付き回答を生成します。会話履歴と検索パラメータに基づき、ナレッジベースは以下を実行します。

1. 会話全体を分析し、ユーザーの情報ニーズを推測
1. 複合クエリを焦点の絞られたサブクエリに分解
1. ナレッジソースに対してサブクエリを並行して実行
1. セマンティックランカーを使用して結果を再ランク付けし、フィルタリング
1. 上位の結果を自然言語の回答に統合（ANSWER_SYNTHESISの場合）

- Reasoning effort を設定する
    - https://learn.microsoft.com/azure/search/agentic-retrieval-how-to-set-retrieval-reasoning-effort

In [ ]:
import textwrap

agent_client = KnowledgeBaseRetrievalClient(endpoint=endpoint, knowledge_base_name=knowledge_base_name, credential=credential)

messages.append({
    "role": "user",
    "content": """
平清盛の生まれた場所はどこですか？
   """
})

req = KnowledgeBaseRetrievalRequest(
    messages=[
        KnowledgeBaseMessage(
            role=m["role"],
            content=[KnowledgeBaseMessageTextContent(text=m["content"])]
        ) for m in messages if m["role"] != "system"
    ],
    knowledge_source_params=[
        SearchIndexKnowledgeSourceParams(
            knowledge_source_name=knowledge_source_name,
            include_references=True,
            include_reference_source_data=True,
            always_query_source=True,
            
        )
    ],
    include_activity=True,
    retrieval_reasoning_effort=KnowledgeRetrievalMediumReasoningEffort()
)

retrieval_result = agent_client.retrieve(retrieval_request=req)
print("Response")
print(textwrap.fill(retrieval_result.response[0].content[0].text, width=120))

## 結果の評価

https://learn.microsoft.com/azure/search/agentic-retrieval-how-to-retrieve

### References:
`references` 配列は、基になるグラウンドデータからの直接参照であり、応答の生成に使用される `sourceData` が含まれています。これは Agentic retrieval エンジンによって検出され、セマンティックランキングされたすべての単一のドキュメントで構成されます。 


In [ ]:
import json
print("## Activity: ")

# Activity -> JSON string of activity as list of dicts
if retrieval_result.activity:
    print(json.dumps([a.as_dict() for a in retrieval_result.activity], indent=2, ensure_ascii=False))
else:
    print("No activity found on 'result'")

### Activity:
`activity` 配列はクエリープランを出力します。これによりリクエストの実行時に行われた操作を追跡するのに役立ちます。また、リソース呼び出しの課金への影響と頻度を理解できるように、運用上の透明性も提供されます。

In [ ]:

print("## References:")
if retrieval_result.references:
    print(json.dumps([r.as_dict() for r in retrieval_result.references[:3]], indent=2, ensure_ascii=False))
else:
    print("No references found on 'result'")
    
